In [9]:

from openai import OpenAI
import os
import pandas as pd
import jsonlines
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from tqdm import tqdm
import json, csv
load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('OPENAI_API_KEY')
client = OpenAI()

In [10]:
class ArgumentClassification(BaseModel):
    id: str = Field(description="The ID the comment being analyzed")    
    label: int = Field(description="The label associated with the argument (0 or 1)") 

## Gay Marriage

In [11]:
def classify_text_gpt(id: str, comment_text: str, argument: str) -> dict:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
        {"role": "system", "content": f"""
        Analyze the given comment about gay marriage in relation to a specific argument. Your need to:
        Identify if the comment makes use the given argument. If it does, assign the label 1. If it does not, assign the label 0.
        Do NOT use any other label.
        Do NOT include the comment or the argument in the response.
        
        The argument to analyze is: {argument}
        
        Provide your response in the following JSON format:
        
        {{
            "id": "{id}",
            "label": "the label for the use of the argument in the comment"
        }}
        
        Analyze the following comment in relation to the given argument
        """},
        {"role": "user", "content": comment_text},
        ],
        response_format=ArgumentClassification,
        temperature=0,
        top_p=1,
    )

    return completion.choices[0].message.content

In [12]:
gm = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/GM_all_arguments.csv')

def process_comments_with_arguments(df: pd.DataFrame, output_file: str):
    with jsonlines.open(output_file, mode='a') as writer:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id']
            comment_text = row['comment_text']
            argument_text = row['argument_text']

            try:
                classification = classify_text_gpt(comment_id, comment_text, argument_text)
                classification = json.loads(classification)
                output_entry = {"id": comment_id, "label": classification["label"]}
                #print(output_entry)
                writer.write(output_entry)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                continue

output_file = 'comarg_gm_argument_identification.jsonl'
process_comments_with_arguments(gm, output_file)

Processing comments: 100%|██████████| 1386/1386 [22:05<00:00,  1.05it/s]


In [13]:
# Under God in Pledge

def classify_text_ugip(id: str, comment_text: str, argument: str) -> dict:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
        {"role": "system", "content": f"""
        Analyze the given comment in relation to a specific argument about whether "Under God" should appear in the US Pledge of Allegiance.
        Identify if the comment makes use the given argument. If it does, assign the label 1. If it does not, assign the label 0.
        Do NOT use any other label.
        Do NOT include the comment or the argument in the response.
        
        The argument to analyze is: {argument}
        
        Provide your response in the following JSON format:
        
        {{
            "id": "{id}",
            "label": "the label for the use of the argument in the comment"
        }}
        
        Analyze the following comment in relation to the given argument
        """},
        {"role": "user", "content": comment_text},
        ],
        response_format=ArgumentClassification,
        temperature=0,
        top_p=1,
    )

    return completion.choices[0].message.content

In [14]:
ugip = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/UGIP_all_arguments.csv')

def process_comments_with_arguments(df: pd.DataFrame, output_file: str) -> List[dict]:
    with jsonlines.open(output_file, mode='a') as writer:
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
            comment_id = row['id'] 
            comment_text = row['comment_text']  
            argument_text = row['argument_text']

            try:
                classification = classify_text_ugip(comment_id, comment_text, argument_text)
                classification = json.loads(classification)
                output_entry = {"id": comment_id, "label": classification["label"]}
                #print(output_entry)
                writer.write(output_entry)
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
            continue


process_comments_with_arguments(ugip, 'comarg_ugip_argument_identification_gpt.jsonl')

Processing comments:   0%|          | 0/2100 [00:00<?, ?it/s]

Processing comments: 100%|██████████| 2100/2100 [32:29<00:00,  1.08it/s] 
